In [ ]:
%tensorflow_version 2.x

In [ ]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

## Step 1:**Downloading the data set**

1.   unzipping the data set
2.   Loading the data into data frame as follows


*   questions_df
*   answers_df
*   tags_df






In [ ]:
!wget https://www.dropbox.com/s/5721wcs2guuykzl/stacksample.zip?dl=0

--2021-03-17 03:51:17--  https://www.dropbox.com/s/5721wcs2guuykzl/stacksample.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.18, 2620:100:6030:18::a27d:5012
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/5721wcs2guuykzl/stacksample.zip [following]
--2021-03-17 03:51:17--  https://www.dropbox.com/s/raw/5721wcs2guuykzl/stacksample.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc9a5738045045b2b6eff4430f02.dl.dropboxusercontent.com/cd/0/inline/BK2Vj8icZlLXf6D-B9j0gYiMHsS5WI2Gu3CACGfP7gbgE-qugydilvwFrgfQxOerPQQGvx-vNJydq82L3pM9KdrFb96nlEnsgwYrJtJ3ZOP3XCq_ILSbg7a-YU0VUKciEFpsj42F8kPeZ95E4Lpvgb77/file# [following]
--2021-03-17 03:51:18--  https://uc9a5738045045b2b6eff4430f02.dl.dropboxusercontent.com/cd/0/inline/BK2Vj8icZlLXf6D-B9j0gYiMHsS5WI2Gu3CACGfP7gbgE-qugydilvwFrgfQxOerPQQGvx-vNJy

In [ ]:
# unzipping the data set

In [ ]:
!unzip '/content/stacksample.zip?dl=0'

Archive:  /content/stacksample.zip?dl=0
  inflating: Answers.csv             
  inflating: Questions.csv           
  inflating: Tags.csv                


In [ ]:
# questions=r'/content/Questions.csv'
# answers=r'/content/Answers.csv'
# tags=r'/content/Tags.csv'

In [ ]:
questions_df=pd.read_csv('../content/Questions.csv',encoding='ISO-8859-1')
answers_df=pd.read_csv('../content/Answers.csv',encoding='ISO-8859-1')
tags_df=pd.read_csv('../content/Tags.csv',encoding='ISO-8859-1')

In [ ]:
# Common encoding used are(try following and see which one suits the data)
# UTF-8
# Latin-1 (also known as ISO-8859-1)
# Windows-1251

In [ ]:
questions_df.head(n=2)

,Id,OwnerUserId,CreationDate,ClosedDate,Score,Title,Body
0,80,26.0,2008-08-01T13:57:07Z,NaN,26,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...
1,90,58.0,2008-08-01T14:41:24Z,2012-12-26T03:45:49Z,144,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...


In [ ]:
answers_df.head(n=2)

,Id,OwnerUserId,CreationDate,ParentId,Score,Body
0,92,61.0,2008-08-01T14:45:37Z,90,13,"<p><a href=""http://svnbook.red-bean.com/"">Vers..."
1,124,26.0,2008-08-01T16:09:47Z,80,12,<p>I wound up using this. It is a kind of a ha...


In [ ]:
tags_df.head(n=2)

,Id,Tag
0,80,flex
1,80,actionscript-3


## Step 2: **Reducing the problem statement into top 10 tags**

In [ ]:
grouped_tags=tags_df.groupby('Tag').size().reset_index(name='count')

In [ ]:
grouped_tags.head(n=2)

,Tag,count
0,.a,13
1,.app,14


In [ ]:
grouped_tags.Tag.describe()

count        37034
unique       37034
top       hyperref
freq             1
Name: Tag, dtype: object

In [ ]:
#Here we are reducing the problem to the  most common tags in the dataset)
no_classes =10
grouped_tags_copy=grouped_tags.copy()
top_10_tags=grouped_tags_copy.nlargest(no_classes,columns="count")


In [ ]:
top_10_tags

,Tag,count
16121,javascript,124155
16020,java,115212
4301,c#,101186
24211,php,98808
979,android,90659
16611,jquery,78542
25712,python,64601
14180,html,58976
4311,c++,47591
15565,ios,47009


In [ ]:
#filtering the top 10 tags data 
tags_df.Tag=tags_df.Tag.apply(lambda tag : tag if tag in top_10_tags.Tag.values else None)

In [ ]:
tags_df

,Id,Tag
0,80,None
1,80,None
2,80,None
3,90,None
4,90,None
...,...,...
3750989,40143360,javascript
3750990,40143360,None
3750991,40143380,None
3750992,40143380,None


In [ ]:
tags_df.dropna(inplace=True)

In [ ]:
tags_df

,Id,Tag
14,260,c#
18,330,c++
28,650,c#
35,930,c#
39,1010,c#
...,...,...
3750974,40143150,javascript
3750981,40143190,python
3750984,40143210,php
3750987,40143340,android


## Step 3: **Data Pre-processesing**

1. Preparing the contents of data frame 

*   Removing the html tags of Body column in questions_df
*   Merging the Title and Body column in question_df
* Filtering the Top 10 taged questions_df and merging the question_df & tags_df
*Droping ununcessary columns


2.   Label Creation

*   Tokenization

3.   Spliting the data into Train & Test 












In [ ]:
#preparing the contents of dataframe
# to remove the html tags
import re

def strip_html_tags(body):
  regex = re.compile('<.*?>')
  return re.sub(regex,'',body)

In [ ]:
questions_df.head(n=2)

,Id,OwnerUserId,CreationDate,ClosedDate,Score,Title,Body
0,80,26.0,2008-08-01T13:57:07Z,NaN,26,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...
1,90,58.0,2008-08-01T14:41:24Z,2012-12-26T03:45:49Z,144,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...


In [ ]:
questions_df.columns

Index(['Id', 'OwnerUserId', 'CreationDate', 'ClosedDate', 'Score', 'Title',
       'Body'],
      dtype='object')

In [ ]:
questions_df['Body']=questions_df.Body.apply(strip_html_tags)

In [ ]:
questions_df['Body']

0          I've written a database generation script in S...
1          Are there any really good tutorials explaining...
2          Has anyone got experience creating SQL-based A...
3          This is something I've pseudo-solved many time...
4          I have a little game written in C#. It uses a ...
                                 ...                        
1264211    I am building a custom MVC project and I have ...
1264212    The API docs show that you should be able to m...
1264213    Under minifyEnabled I changed from false to tr...
1264214    I have input which I use to filter my array of...
1264215    I'm running a mocha test and I noticed my chan...
Name: Body, Length: 1264216, dtype: object

In [ ]:
questions_df['Text']=questions_df['Title']+' '+questions_df['Body']

In [ ]:
questions_df['Text']

0          SQLStatement.execute() - multiple queries in o...
1          Good branching and merging tutorials for Torto...
2          ASP.NET Site Maps Has anyone got experience cr...
3          Function for creating color wheels This is som...
4          Adding scripting functionality to .NET applica...
                                 ...                        
1264211    URL routing in PHP (MVC) I am building a custo...
1264212    Bigquery.Jobs.Insert - Resumable Upload? The A...
1264213    Obfuscating code in android studio Under minif...
1264214    How to fire function after v-model change? I h...
1264215    npm run mocha test - files being cached I'm ru...
Name: Text, Length: 1264216, dtype: object

In [ ]:
questions_df.shape

(1264216, 8)

In [ ]:
tags_df.shape

(826739, 2)

In [ ]:
tags_df.head()

,Id,Tag
14,260,c#
18,330,c++
28,650,c#
35,930,c#
39,1010,c#


In [ ]:
questions_df.sort_values('Id').head()

,Id,OwnerUserId,CreationDate,ClosedDate,Score,Title,Body,Text
0,80,26.0,2008-08-01T13:57:07Z,NaN,26,SQLStatement.execute() - multiple queries in o...,I've written a database generation script in S...,SQLStatement.execute() - multiple queries in o...
1,90,58.0,2008-08-01T14:41:24Z,2012-12-26T03:45:49Z,144,Good branching and merging tutorials for Torto...,Are there any really good tutorials explaining...,Good branching and merging tutorials for Torto...
2,120,83.0,2008-08-01T15:50:08Z,NaN,21,ASP.NET Site Maps,Has anyone got experience creating SQL-based A...,ASP.NET Site Maps Has anyone got experience cr...
3,180,2089740.0,2008-08-01T18:42:19Z,NaN,53,Function for creating color wheels,This is something I've pseudo-solved many time...,Function for creating color wheels This is som...
4,260,91.0,2008-08-01T23:22:08Z,NaN,49,Adding scripting functionality to .NET applica...,I have a little game written in C#. It uses a ...,Adding scripting functionality to .NET applica...


In [ ]:
tags_df.sort_values('Id').head()

,Id,Tag
14,260,c#
18,330,c++
28,650,c#
35,930,c#
39,1010,c#


In [ ]:
tags_df.nunique()

Id     706336
Tag        10
dtype: int64

In [ ]:
tags_Ids=list(tags_df['Id'])

In [ ]:
questions_df_top_10=questions_df[questions_df['Id'].isin(tags_Ids)]

In [ ]:
questions_df_top_10

,Id,OwnerUserId,CreationDate,ClosedDate,Score,Title,Body,Text
4,260,91.0,2008-08-01T23:22:08Z,NaN,49,Adding scripting functionality to .NET applica...,I have a little game written in C#. It uses a ...,Adding scripting functionality to .NET applica...
5,330,63.0,2008-08-02T02:51:36Z,NaN,29,Should I use nested classes in this case?,I am working on a collection of classes used f...,Should I use nested classes in this case? I am...
8,650,143.0,2008-08-03T11:12:52Z,NaN,79,Automatically update version number,I would like the version property of my applic...,Automatically update version number I would li...
10,930,245.0,2008-08-04T00:47:25Z,NaN,28,How do I connect to a database and loop over a...,What's the simplest way to connect and query a...,How do I connect to a database and loop over a...
11,1010,67.0,2008-08-04T03:59:42Z,NaN,14,"How to get the value of built, encoded ViewState?",I need to grab the base64-encoded representati...,"How to get the value of built, encoded ViewSta..."
...,...,...,...,...,...,...,...,...
1264208,40143150,5496690.0,2016-10-19T23:31:41Z,NaN,0,"YouTube iFrame API: no ready call, no error call","I'm trying to detect the ""flash out of date"" e...","YouTube iFrame API: no ready call, no error ca..."
1264210,40143190,333403.0,2016-10-19T23:36:01Z,NaN,1,How to execute multiline python code from a ba...,I need to extend a shell script (bash). As I a...,How to execute multiline python code from a ba...
1264211,40143210,5610777.0,2016-10-19T23:38:01Z,NaN,0,URL routing in PHP (MVC),I am building a custom MVC project and I have ...,URL routing in PHP (MVC) I am building a custo...
1264213,40143340,7028647.0,2016-10-19T23:52:50Z,NaN,1,Obfuscating code in android studio,Under minifyEnabled I changed from false to tr...,Obfuscating code in android studio Under minif...


In [ ]:
#merging the data(questions_df_top_10 & tags_df)
questions_df_top_10=questions_df_top_10.merge(tags_df)

In [ ]:
questions_df_top_10.head()

,Id,OwnerUserId,CreationDate,ClosedDate,Score,Title,Body,Text,Tag
0,260,91.0,2008-08-01T23:22:08Z,NaN,49,Adding scripting functionality to .NET applica...,I have a little game written in C#. It uses a ...,Adding scripting functionality to .NET applica...,c#
1,330,63.0,2008-08-02T02:51:36Z,NaN,29,Should I use nested classes in this case?,I am working on a collection of classes used f...,Should I use nested classes in this case? I am...,c++
2,650,143.0,2008-08-03T11:12:52Z,NaN,79,Automatically update version number,I would like the version property of my applic...,Automatically update version number I would li...,c#
3,930,245.0,2008-08-04T00:47:25Z,NaN,28,How do I connect to a database and loop over a...,What's the simplest way to connect and query a...,How do I connect to a database and loop over a...,c#
4,1010,67.0,2008-08-04T03:59:42Z,NaN,14,"How to get the value of built, encoded ViewState?",I need to grab the base64-encoded representati...,"How to get the value of built, encoded ViewSta...",c#


In [ ]:
questions_df_top_10.columns

Index(['Id', 'OwnerUserId', 'CreationDate', 'ClosedDate', 'Score', 'Title',
       'Body', 'Text', 'Tag'],
      dtype='object')

In [ ]:
#droping ununcessary columns
questions_df_top_10.drop(['OwnerUserId','CreationDate','ClosedDate','Score','Title',
       'Body'],axis=1,inplace=True)

In [ ]:
questions_df_top_10.head()

,Id,Text,Tag
0,260,Adding scripting functionality to .NET applica...,c#
1,330,Should I use nested classes in this case? I am...,c++
2,650,Automatically update version number I would li...,c#
3,930,How do I connect to a database and loop over a...,c#
4,1010,"How to get the value of built, encoded ViewSta...",c#


In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import MultiLabelBinarizer

In [ ]:
#Label creation 
multiLabelBinarizer = MultiLabelBinarizer()
multiLabelBinarizer.fit([questions_df_top_10.Tag])
multiLabelBinarizer.transform([questions_df_top_10.Tag])

array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])

In [ ]:
Labels= multiLabelBinarizer.classes_
Labels

array(['android', 'c#', 'c++', 'html', 'ios', 'java', 'javascript',
       'jquery', 'php', 'python'], dtype=object)

In [ ]:
y=pd.get_dummies(questions_df_top_10.Tag).to_numpy()

In [ ]:
y.shape

(826739, 10)

In [ ]:
#tokenization
max_len = 180 
max_words = 5000
tokenizer = Tokenizer(num_words=max_words,lower=True)
tokenizer.fit_on_texts(questions_df_top_10.Text)

In [ ]:
# vocab_len =len(tokenizer.index_word.keys())
# vocab_len

In [ ]:
tokenizer

In [ ]:
def get_features(text_series):
  # transforms text data to feature_vectors that can be used in the ml model
  #Tokenizer must be available
  sequences = tokenizer.texts_to_sequences(text_series)
  return pad_sequences(sequences,maxlen=max_len)

def predication_to_label(predication):
  tag_prob = [(labels[i],prob) for i , prob in enumerate(predication.tolist())]
  return dict(sorted(tag_prob,key=lambda kv : kv[1],reverse=True))

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x=get_features(questions_df_top_10.Text)

In [ ]:
x.shape

(826739, 180)

In [ ]:
y.shape

(826739, 10)

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=123)

## Step 4 : **Model building**




*   CNN (Conv1D) is used 






In [ ]:
from tensorflow.keras.layers import  Activation, Dense, Dropout, Input, Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Embedding, Flatten, GlobalMaxPool1D, Dropout, Conv1D
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model




#CNN---Conv1D


In [ ]:
filter_length = 300

model = Sequential()
model.add(Embedding(max_words+1, 20, input_length=max_len))
model.add(Dropout(0.1))
model.add(Conv1D(filter_length, 3, padding='valid', activation='relu', strides=1))
model.add(GlobalMaxPool1D())
model.add(Dense(no_classes))
model.add(Activation('sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['categorical_accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 180, 20)           100020    
_________________________________________________________________
dropout (Dropout)            (None, 180, 20)           0         
_________________________________________________________________
conv1d (Conv1D)              (None, 178, 300)          18300     
_________________________________________________________________
global_max_pooling1d (Global (None, 300)               0         
_________________________________________________________________
dense (Dense)                (None, 10)                3010      
_________________________________________________________________
activation (Activation)      (None, 10)                0         
Total params: 121,330
Trainable params: 121,330
Non-trainable params: 0
__________________________________________________


 **Note** :
  
  If you need more accuracy 
  kindly use vocab_len instead of max_words  
  
  you use following code.
 


 * Vocab_len =len(tokenizer.index_word.keys())
  

In [ ]:
model.fit(x_train,y_train,epochs=100,validation_split=0.2)

Epoch 1/100
16535/16535 [==============================] - 146s 7ms/step - loss: 0.1507 - categorical_accuracy: 0.6838 - val_loss: 0.1113 - val_categorical_accuracy: 0.7665
Epoch 2/100
16535/16535 [==============================] - 112s 7ms/step - loss: 0.1105 - categorical_accuracy: 0.7657 - val_loss: 0.1101 - val_categorical_accuracy: 0.7642
Epoch 3/100
16535/16535 [==============================] - 113s 7ms/step - loss: 0.1082 - categorical_accuracy: 0.7691 - val_loss: 0.1091 - val_categorical_accuracy: 0.7669
Epoch 4/100
16535/16535 [==============================] - 114s 7ms/step - loss: 0.1067 - categorical_accuracy: 0.7725 - val_loss: 0.1083 - val_categorical_accuracy: 0.7693
Epoch 5/100
16535/16535 [==============================] - 111s 7ms/step - loss: 0.1058 - categorical_accuracy: 0.7745 - val_loss: 0.1076 - val_categorical_accuracy: 0.7676
Epoch 6/100
16535/16535 [==============================] - 112s 7ms/step - loss: 0.1050 - categorical_accuracy: 0.7755 - val_loss: 0.10

In [ ]:
y_pred = model.predict_classes(x_test)

In [ ]:
y_pred

array([6, 1, 3, ..., 1, 4, 6])

## Step 5: ***Evaluvation ***

In [ ]:
model.evaluate(x_test,y_test)

5168/5168 [==============================] - 15s 3ms/step - loss: 0.1131 - categorical_accuracy: 0.7574


[0.11309224367141724, 0.7573723196983337]

## **Loss**: 0.1140  , **Categorical_accuracy**: 0.7552 

In [ ]:
# loss: 0.1140 - categorical_accuracy: 0.7552
# [0.11403822153806686, 0.7551950812339783]